In [4]:
import sys
sys.path.append("../../")
import PolyMID
import pandas as pd
from PolyMID.AnalyzeSpectra import fragment_library

In [5]:
file_directory = './'
metabolite_dict,metabolite_list = fragment_library.fragment_library(file_directory=file_directory,Full_NC=False)

    pyruvate_mox_tbdms
    lactate_2tbdms
    alanine_2tbdms
    glycine_2tbdms
    3-hydroxypropionicacid_2tbdms
    valine_2tbdms
    norvaline_2tbdms
    beta-alanine_2tbdms
    leucine_2tbdms
    isoleucine_2tbdms
    succinate_2tbdms
    uracil_2tbdms
    fumarate_2tbdms
    serine_3tbdms
    threonine_3tbdms
    taurine_2tbdms
    methionine_2tbdms
    alpha-ketoglutarate_mox_2tbdms
    malate_3tbdms
    phenylalanine_2tbdms
    aspartate_3tbdms
    proline_MOX_2tbdms
    glutamate_3tbdms
    asparagine_3tbdms
    lysine_3tbdms
    glutamine_3tbdms
    arginine_4tbdms
    citrate_4tbdms
    tyrosine_3tbdms
    histidine_3tbdms
    tryptophan_3tbdms
    glucose
    glucose_clean


In [25]:
Standard_MID_DF = pd.read_csv('Internal_Standards.txt', delimiter='\t', index_col=0)
fragments = Standard_MID_DF.columns

for met_key in metabolite_dict.keys():
    for frag_key in metabolite_dict[met_key]['fragments'].keys():
        if frag_key in fragments:
            Natural_MID = metabolite_dict[met_key]['fragments'][frag_key]['natural_mid'][0:10]
            frag_key_position = Standard_MID_DF.columns.get_loc(frag_key)
            Standard_MID_DF.iloc[0:10,frag_key_position] = Natural_MID

Standard_MID_DF.to_csv('filename.txt', sep='\t', index=True, header=True)